In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from time import sleep
import timeit
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests

pd.set_option('display.max_colwidth', None)
HEADERS = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.129 Safari/537.36"}

In [2]:
driver =  webdriver.Chrome(ChromeDriverManager().install())

[WDM] - Current google-chrome version is 83.0.4103
[WDM] - Get LATEST driver version for 83.0.4103
[WDM] - Get LATEST driver version for 83.0.4103
[WDM] - Trying to download new driver from http://chromedriver.storage.googleapis.com/83.0.4103.39/chromedriver_mac64.zip


[WDM] - Driver has been saved in cache [/Users/hsichengyun/.wdm/drivers/chromedriver/mac64/83.0.4103.39]


In [5]:
url = requests.get('https://flixable.com/netflix-originals/genre/movies/', {'order':'title'}).url
driver.get(url)

In [10]:
# I manually scrolled down to the bottom. (it's just easier than the online code)
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [11]:
# get all netflix org movies till July 1st 2020

list_of_netflix_org_movies = list()
for i in soup.findAll('a',{'class':'title'}):
    if "..." in i.text:
        re = requests.get('https://flixable.com'+i['href'])
        s = BeautifulSoup(re.text, 'html.parser')
        list_of_netflix_org_movies.append(s.find('h1',{'class':'title text-left'}).text)
    else:
        list_of_netflix_org_movies.append(i.text)

In [19]:
print(f"There are {len(list_of_netflix_org_movies)} original movies.\n")
print(f"There are {len(list_of_netflix_org_tvshows)} original tvshows.")

There are 853 original movies.

There are 873 original tvshows.


In [15]:
# I manually scrolled down to the bottom. (it's just easier than the online code)
soup = BeautifulSoup(driver.page_source, 'html.parser')
# get all netflix org tvshows till July 1st 2020

list_of_netflix_org_tvshows = list()
for i in soup.findAll('a',{'class':'title'}):
    if "..." in i.text:
        re = requests.get('https://flixable.com'+i['href'])
        s = BeautifulSoup(re.text, 'html.parser')
        list_of_netflix_org_tvshows.append(s.find('h1',{'class':'title text-left'}).text)
    else:
        list_of_netflix_org_tvshows.append(i.text)

In [27]:
# Make a dataframe and save it
# df = pd.DataFrame()
# df['title'] = list_of_netflix_org_movies
# df['type'] = 'movie'

# df1 = pd.DataFrame()
# df1['title'] = list_of_netflix_org_tvshows
# df1['type'] = 'tvshow'

# df = df.append(df1, ignore_index=True)
# df.to_csv('july2020_org.csv')

In [35]:
while True:
    try:
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        driver.find_element_by_css_selector("#app_mountpoint > div > main > div:nth-child(5) > a > div > button").click()
        sleep(1)
    except:
        break

In [30]:
# get movie links from reelgood
soup = BeautifulSoup(driver.page_source, 'html.parser')

movie_links = list()
rows = soup.findAll('tr',{'class':'css-o6sgwe'})
for row in rows:
    movie_links.append(row.find('td',{'class':'css-1u7zfla e126mwsw1'}).find('a')['href'])

In [33]:
len(movie_links)

3770

In [36]:
# get tvshow links from reelgood
soup = BeautifulSoup(driver.page_source, 'html.parser')

tvshow_links = list()
rows = soup.findAll('tr',{'class':'css-gfsdx9'})
for row in rows:
    tvshow_links.append(row.find('td',{'class':'css-1u7zfla e126mwsw1'}).find('a')['href'])

In [37]:
len(tvshow_links)

2060

In [43]:
movie_links[0:10]

['/movie/1000-mabrouk-2009',
 '/movie/1000-rupee-note-2016',
 '/movie/100-days-of-solitude-2018',
 '/movie/100-meters-2016',
 '/movie/10-days-in-sun-city-2017',
 '/movie/10-jours-en-or-2012',
 '/movie/122-2019',
 '/movie/12-round-gun-2017',
 '/movie/13-cameras-2015',
 '/movie/13-sins-2014']

In [44]:
def collect_data(links):
# links = mlinks.movielinks_rg.tolist()
    start = timeit.default_timer()
    DF = pd.DataFrame(columns=['title','genres','description','year'])
    count = 1
    for link in links:
        if count%10 ==0:
            print(count)
        count += 1
        
        r = requests.get('https://reelgood.com'+link, headers=HEADERS)
        soup = BeautifulSoup(r.text, 'html.parser')
        try:
            title = soup.find('h1',{'itemprop':'name'}).text
        except:
            title = ''
        try:
            temp = soup.findAll('span',{'class':'css-ee2w7g ey4ir3j1'})
            l = []
            for row in temp:
                aTags = row.findAll('a')
                for a in aTags:
                    if 'genre' in a['href'] and a['href'] not in l:
                        l.append(a['href'])
            genres = ','.join([gen.split('/')[-1] for gen in l])
        except:
            genres = ''
        
        description = soup.find('p',{'itemprop':'description'}).text
        year = link.split('-')[-1]
        
        df = pd.DataFrame(columns=['title','genres','description','year'], data=[{
            'title' : title,
            'genres' : genres,
            'description': description,
            'year' : year
        }])
        DF = DF.append(df, ignore_index=True)
        sleep(0.25)
    stop = timeit.default_timer()
    t =int((stop-start)/60)
    print(f"It took {t} minutes to get 100 genres.")   
    return DF

In [45]:
test = collect_data(movie_links[0:3])

It took 0 minutes to get 100 genres.


In [46]:
test

,title,genres,description,year
0,1000 Mabrouk,"comedy,drama,fantasy,thriller","About a man who wakes up one morning to discover that the day repeats, a carbon copy of the day before, trapping him in an inescapable vicious circle. This man has a dream to live for 100 years, however he is living for one day only and that day keep repeating to show him the negative and positive things in his life.1000 Mabrouk featuring Ahmed Helmy and Laila Ezz El Arab is streaming with subscription on Netflix. It's a comedy and drama movie with a better than average IMDb audience rating of 7.4 (3,427 votes).",2009
1,1000 Rupee Note,drama,"Poor Parobudhi receives a thousand rupee note from a politician at an election rally. However, she has no idea about the kind of trouble she just got into. Ek Hazarachi Note is an encouraging tale of humanity against money.1000 Rupee Note featuring Sandeep Pathak is streaming with subscription on Netflix, streaming with subscription on Kanopy, available for rent or purchase on iTunes, and 4 others. It's a drama movie with a high Rotten Tomatoes (critics) score of 80% and a better than average IMDb audience rating of 7.3 (331 votes).",2016
2,100 Days of Loneliness,"documentary,drama",An extraordinary and touching tale about a man alone in the nature.100 Days of Loneliness featuring José Díaz is streaming with subscription on Netflix. It's a documentary and drama movie with a better than average IMDb audience rating of 6.7 (267 votes).,2018


In [47]:
ix = 100

for i in range(37): #just run 37 times. The rest 70 i will get by other codes
    print(ix)
    try:
        if i == 0:
            movieDF = collect_data(movie_links[0:ix]) # first 100
            movieDF.to_csv('movieDF.csv') # save file
            ix += 100
        else:
            temp = collect_data(movie_links[(ix-100):ix]) # the next 100
            movieDF = movieDF.append(temp, ignore_index=True) # add to the df
            movieDF.to_csv('movieDF.csv') # save file
            ix += 100 
    except:
        print(i)
        break

100
10
20
30
40
50
60
70
80
90
100
It took 2 minutes to get 100 genres.
200
10
20
30
40
50
60
70
80
90
100
It took 3 minutes to get 100 genres.
300
10
20
30
40
50
60
70
80
90
100
It took 3 minutes to get 100 genres.
400
10
20
30
40
50
60
70
80
90
100
It took 3 minutes to get 100 genres.
500
10
20
30
40
50
60
70
80
90
100
It took 3 minutes to get 100 genres.
600
10
20
30
40
50
60
70
80
90
100
It took 3 minutes to get 100 genres.
700
10
20
30
40
50
60
70
80
90
100
It took 2 minutes to get 100 genres.
800
10
20
30
40
50
60
70
80
90
100
It took 2 minutes to get 100 genres.
900
10
20
30
40
50
60
70
80
90
100
It took 2 minutes to get 100 genres.
1000
10
20
30
40
50
60
70
80
90
100
It took 3 minutes to get 100 genres.
1100
10
20
30
40
50
60
70
80
90
100
It took 2 minutes to get 100 genres.
1200
10
20
30
40
50
60
70
80
90
100
It took 2 minutes to get 100 genres.
1300
10
20
30
40
50
60
70
80
90
100
It took 2 minutes to get 100 genres.
1400
10
20
30
40
50
60
70
80
90
100
It took 2 minutes to get

In [48]:
temp = collect_data(movie_links[3700:]) # the next 100
movieDF = movieDF.append(temp, ignore_index=True) # add to the df
movieDF.to_csv('movieDF.csv') # save file

10
20
30
40
50
60
70
It took 2 minutes to get 100 genres.


In [52]:
ix = 100

for i in range(20): #just run 20 times. The rest 60 i will get by other codes
    print(ix)
    try:
        if i == 0:
            tvshowDF = collect_data(tvshow_links[0:ix]) # first 100
            tvshowDF.to_csv('tvshowDF.csv') # save file
            ix += 100
        else:
            temp = collect_data(tvshow_links[(ix-100):ix]) # the next 100
            tvshowDF = tvshowDF.append(temp, ignore_index=True) # add to the df
            tvshowDF.to_csv('tvshowDF.csv') # save file
            ix += 100 
    except:
        print(i)
        break

temp = collect_data(tvshow_links[2000:]) # the next 100
tvshowDF = tvshowDF.append(temp, ignore_index=True) # add to the df
tvshowDF.to_csv('tvshowDF.csv') # save file

100
10
20
30
40
50
60
70
80
90
100
It took 2 minutes to get 100 genres.
200
10
20
30
40
50
60
70
80
90
100
It took 3 minutes to get 100 genres.
300
10
20
30
40
50
60
70
80
90
100
It took 4 minutes to get 100 genres.
400
10
20
30
40
50
60
70
80
90
100
It took 4 minutes to get 100 genres.
500
10
20
30
40
50
60
70
80
90
100
It took 5 minutes to get 100 genres.
600
10
20
30
40
50
60
70
80
90
100
It took 4 minutes to get 100 genres.
700
10
20
30
40
50
60
70
80
90
100
It took 4 minutes to get 100 genres.
800
10
20
30
40
50
60
70
80
90
100
It took 3 minutes to get 100 genres.
900
10
20
30
40
50
60
70
80
90
100
It took 3 minutes to get 100 genres.
1000
10
20
30
40
50
60
70
80
90
100
It took 2 minutes to get 100 genres.
1100
10
20
30
40
50
60
70
80
90
100
It took 4 minutes to get 100 genres.
1200
10
20
30
40
50
60
70
80
90
100
It took 5 minutes to get 100 genres.
1300
10
20
30
40
50
60
70
80
90
100
It took 5 minutes to get 100 genres.
1400
10
20
30
40
50
60
70
80
90
100
It took 4 minutes to get

In [113]:
# Put them altogether
movieDF['type'] = 'movie'
tvshowDF['type'] = 'tvshow'
movie_tv_df = movieDF.append(tvshowDF, ignore_index=True)

In [71]:
movie_tv_df.head(1)

,title,genres,description,year,type
0,1000 Mabrouk,"comedy,drama,fantasy,thriller","About a man who wakes up one morning to discover that the day repeats, a carbon copy of the day before, trapping him in an inescapable vicious circle. This man has a dream to live for 100 years, however he is living for one day only and that day keep repeating to show him the negative and positive things in his life.1000 Mabrouk featuring Ahmed Helmy and Laila Ezz El Arab is streaming with subscription on Netflix. It's a comedy and drama movie with a better than average IMDb audience rating of 7.4 (3,427 votes).",2009,movie


In [115]:
# understand the repeatitive title names
d = {}
for title in movie_tv_df.title.tolist():
    if title not in d.keys():
        d[title] = 1
    else:
        d[title] += 1

# listOfRepeat = []
for k, v in d.items():
    if v != 1:
#         listOfRepeat.append(k)
        print(k, v)

In [95]:
def get_index(t):
    ix = movie_tv_df.loc[movie_tv_df['title'] == t].index
    return ix

In [112]:
idx = get_index('Aftermath') # the second one 
# movie_tv_df.loc[idx, 'title'] + '_' +movie_tv_df.loc[idx, 'year']
i1 = idx[1]
# i2 = idx[2]
print(i1, idx, len(idx))

3809 Int64Index([124, 3809], dtype='int64') 2


In [114]:
for rep in listOfRepeat:
    if len(get_index(rep)) == 2:
        idx = get_index(rep)[1]
        movie_tv_df.loc[idx, 'title'] = movie_tv_df.loc[idx, 'title'] + '_' +movie_tv_df.loc[idx, 'year']
    else:
        try:
            idx = get_index(rep)
            idx1 = idx[1]
            idx2 = idx[2]
            movie_tv_df.loc[idx1, 'title'] = movie_tv_df.loc[idx1, 'title'] + '_' +movie_tv_df.loc[idx1, 'year']
            movie_tv_df.loc[idx2, 'title'] = movie_tv_df.loc[idx2, 'title'] + '_' +movie_tv_df.loc[idx2, 'year']
        except:
            print(rep)

In [117]:
movie_tv_df.to_csv('dataCollection.csv')